In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [2]:
#Loading dataset values
dfrm = pd.read_csv(r"C:\\Gokul\Cyber_Security_Space_Data.csv")

In [3]:
dfrm.head()

,objid,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
0,1.237650e+18,183.531326,0.089693,19.47406,17.04240,15.94699,15.50342,15.22531,752,301,4,267,3.722360e+18,STAR,-0.000009,3306,54922,491
1,1.237650e+18,183.598370,0.135285,18.66280,17.21449,16.67637,16.48922,16.39150,752,301,4,267,3.638140e+17,STAR,-0.000055,323,51615,541
2,1.237650e+18,183.680207,0.126185,19.38298,18.19169,17.47428,17.08732,16.80125,752,301,4,268,3.232740e+17,GALAXY,0.123111,287,52023,513
3,1.237650e+18,183.870529,0.049911,17.76536,16.60272,16.16116,15.98233,15.90438,752,301,4,269,3.722370e+18,STAR,-0.000111,3306,54922,510
4,1.237650e+18,183.883288,0.102557,17.55025,16.26342,16.43869,16.55492,16.61326,752,301,4,269,3.722370e+18,STAR,0.000590,3306,54922,512


In [4]:
def dependent_transform(form):
    if form=='GALAXY':
        return 0
    elif form=='STAR':
        return 1
    else:
        return 2

In [5]:
#Implementing the above function
dfrm['form'] = dfrm['class'].apply(dependent_transform)

In [6]:
#Initialising the independent and the dependent variable
indp = dfrm.drop(['class','form'], axis=1)
dp = dfrm['form']

In [8]:
#Forming the training and test set
X_train, X_test, Y_train, Y_test = train_test_split(indp, dp, test_size=0.20, random_state=100)

In [10]:
#Tensflor feature variable conversionn
feat_tensor = [
            tf.feature_column.numeric_column('dec'),
            tf.feature_column.numeric_column('u'),
            tf.feature_column.numeric_column('g'),
            tf.feature_column.numeric_column('ra'),
            tf.feature_column.numeric_column('i'),
            tf.feature_column.numeric_column('mjd'),
            tf.feature_column.numeric_column('plate'),
            tf.feature_column.numeric_column('redshift'),
            tf.feature_column.numeric_column('fiberid'),
            tf.feature_column.numeric_column('r'),
            tf.feature_column.numeric_column('z')
           ]

In [11]:
#Deep Neural Network parameter definition
def dnn_input(n_epochs,n_batches,shuffle):
    return tf.compat.v1.estimator.inputs.pandas_input_fn(
        x=X_train,
        y=Y_train,
        num_epochs=n_epochs,
        shuffle=shuffle,
        batch_size=n_batches
    )

In [12]:
#Deep Neural Network model validation
def dnn_vldtn(n_epochs,n_batches,shuffle):
    return tf.compat.v1.estimator.inputs.pandas_input_fn(
        x=X_test,
        y=Y_test,
        num_epochs=n_epochs,
        shuffle=shuffle,
        batch_size=n_batches
    )

In [13]:
#Tensorflow model for a Deep Neural Network
dp_nrl_mdl = tf.estimator.DNNClassifier(n_classes=3,
                                       feature_columns=feat_tensor,
                                       activation_fn=tf.nn.relu,
                                       optimizer='Adam',
                                       hidden_units=[1024,512,256,32,3],
                                       dropout=0.25
                                      )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\gokul\\AppData\\Local\\Temp\\tmp8q93stdc', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [15]:
#Training the Tensorflow Deep Neural Network Model
dp_nrl_mdl.train(input_fn=dnn_input(100,128,True),steps=1400)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\gokul\AppData\Local\Temp\tmp8q93stdc\model.ckpt-700
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 700...
INFO:tensorflow:Saving checkpoints for 700 into C:\Users\gokul\AppData\Local\Temp\tmp8q93stdc\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 700...
INFO:tensorflow:loss = 0.9267006, step = 700
INFO:tensorflow:global_step/sec: 70.7519
INFO:tensorflow:loss = 0.90640473, step = 800 (1.421 sec)
INFO:tensorflow:global_step/sec: 63.4234
INFO:tensorflow:loss = 0.8947601, step = 900 (1.571 sec)
INFO:tensorflow:global_step/sec: 73.2971
INFO:tensorflow:loss = 0.8773994, step = 1000 (1

In [21]:
#Output for Tensorflow Deep Neural Network Model
dp_nrl_mdl.evaluate(input_fn=dnn_vldtn(100,128,True),steps=1400)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2023-02-26T20:51:46
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\gokul\AppData\Local\Temp\tmp8q93stdc\model.ckpt-2100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [140/1400]
INFO:tensorflow:Evaluation [280/1400]
INFO:tensorflow:Evaluation [420/1400]
INFO:tensorflow:Evaluation [560/1400]
INFO:tensorflow:Evaluation [700/1400]
INFO:tensorflow:Evaluation [840/1400]
INFO:tensorflow:Evaluation [980/1400]
INFO:tensorflow:Evaluation [1120/1400]
INFO:tensorflow:Evaluation [1260/1400]
INFO:tensorflow:Evaluation [1400/1400]
INFO:tensorflow:Inference Time : 6.86432s
INFO:tensorflow:Finished evaluation at 2023-02-26-20:51:53
INFO:tensorflow:Saving dict for global step 2100: accuracy = 0.49954242, average_loss = 0.92214197, global_step = 2100, loss = 0.92214197
INFO:tensorflow:Saving

{'accuracy': 0.49954242,
 'average_loss': 0.92214197,
 'loss': 0.92214197,
 'global_step': 2100}

In [17]:
#Implementation of combination of Deep Neural Network and Linear Classifier model
dnn_lnr_mdl = tf.estimator.DNNLinearCombinedClassifier(n_classes=3,dnn_optimizer='Adam',dnn_dropout=0.25,dnn_hidden_units=[1024,512,256,32,3],dnn_activation_fn='relu',dnn_feature_columns=feat_tensor,linear_feature_columns=feat_tensor)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\gokul\\AppData\\Local\\Temp\\tmpp_72eaq3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [18]:
#Training the combination of Tensorflow Deep Neural Network Model and Linear Classifier model
dnn_lnr_mdl.train(input_fn=dnn_input(100,128,True),steps=1400)

INFO:tensorflow:Calling model_fn.


C:\Users\gokul\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\gokul\AppData\Local\Temp\tmpp_72eaq3\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 697.7014, step = 0
INFO:tensorflow:global_step/sec: 59.1157
INFO:tensorflow:loss = 42.70555, step = 100 (1.692 sec)
INFO:tensorflow:global_step/sec: 76.6015
INFO:tensorflow:loss = 13.158686, step = 200 (1.305 sec)
INFO:tensorflow:global_step/sec: 78.0024
INFO:tensorflow:loss = 3.8514614, step = 300 (1.282 sec)
INFO:tensorflow:global_step/sec: 70.5401
INFO:tensorflow:loss = 2.8376093, step = 400 (1.41

In [19]:
#Output for combination of Tensorflow Deep Neural Network Model and Linear Classifier model
dnn_lnr_mdl.evaluate(input_fn=dnn_vldtn(100,128,True),steps=1400)

INFO:tensorflow:Calling model_fn.


C:\Users\gokul\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2023-02-26T12:59:10
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\gokul\AppData\Local\Temp\tmpp_72eaq3\model.ckpt-1400
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [140/1400]
INFO:tensorflow:Evaluation [280/1400]
INFO:tensorflow:Evaluation [420/1400]
INFO:tensorflow:Evaluation [560/1400]
INFO:tensorflow:Evaluation [700/1400]
INFO:tensorflow:Evaluation [840/1400]
INFO:tensorflow:Evaluation [980/1400]
INFO:tensorflow:Evaluation [1120/1400]
INFO:tensorflow:Evaluation [1260/1400]
INFO:tensorflow:Evaluation [1400/1400]
INFO:tensorflow:Inference Time : 7.32339s
INFO:tensorflow:Finished evaluation at 2023-02-26-12:59:17
INFO:tensorflow:Saving dict for global step 1400: accuracy = 0.7964453, average_loss = 2.1586633, global_step = 1400, loss = 2.1586633
INFO:tensorflow:Saving 'checkpoint_path' summary for global

{'accuracy': 0.7964453,
 'average_loss': 2.1586633,
 'loss': 2.1586633,
 'global_step': 1400}